# Test Folium interactive maps
* total unsheltered pop
* 2017, 2018, 2019 individual year maps

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import intake
import json
from branca.colormap import linear
import folium

In [2]:
catalog = intake.open_catalog('../catalogs/*.yml')
bucket_name = 's3://city-of-los-angeles-data-lake/public-health-dashboard/'

## Prep df to map

In [3]:
df = gpd.read_file(f'{bucket_name}gis/intermediate/homelessness_care_tracts.geojson')
df.rename(columns = {'GEOID': 'id'}, inplace = True)
df = df.set_index('id')
df.head()

,SPA,SD,CD,year,unsheltered,sheltered,tot_homeless,bulky,encampment,illegal,other,pop,full_area,clipped_area,geometry
id,,,,,,,,,,,,,,,
06037139301,2,3,3,2017,16.156,0.0,16.156,1,28,1,0,4603,1.106305,1.106305,"POLYGON ((-118.57150 34.17758, -118.57148 34.1..."
06037139301,2,3,3,2018,1.000,0.0,1.000,0,5,4,0,4603,1.106305,1.106305,"POLYGON ((-118.57150 34.17758, -118.57148 34.1..."
06037139301,2,3,3,2019,4.315,0.0,4.315,6,54,0,0,4603,1.106305,1.106305,"POLYGON ((-118.57150 34.17758, -118.57148 34.1..."
06037139302,2,3,3,2017,4.021,0.0,4.021,0,5,0,0,5389,0.130598,0.130598,"POLYGON ((-118.54073 34.18019, -118.54070 34.1..."
06037139302,2,3,3,2018,3.000,0.0,3.000,0,2,0,0,5389,0.130598,0.130598,"POLYGON ((-118.54073 34.18019, -118.54070 34.1..."


## Total unsheltered over 3 years

In [4]:
df1 = df.dissolve(by = ['id', 'clipped_area', 'full_area', 
                        'CD', 'pop', 'SPA', 'SD'], aggfunc = 'sum').reset_index().drop(columns = ['year'])

# to_json converts the gdf to json. json.loads converts it into dictionary 
# geo_data can have many more columns than just geometry. Need to include other columns if you want to have the HTML popup include info.
geo_data = json.loads(df1.set_index('id').to_json())

# Take what we want to map and turn it into a dictionary
# Can only include the key-value pair, the value you want to map, nothing more.
tot_unsheltered = dict(zip(df1['id'].tolist(), df1['unsheltered'].tolist()))

df1.unsheltered.describe()

count     994.000000
mean       71.426540
std       162.045017
min         0.000000
25%        18.554500
50%        39.450000
75%        76.858750
max      3823.916000
Name: unsheltered, dtype: float64

In [5]:
m = folium.Map(
    location=[34.0536, -118.2427], 
    zoom_start=12, 
    tiles='cartodbpositron'
)

bins = [0, 50, 100, 500, 4000]

unsheltered = folium.Choropleth(
    geo_data = df1,
    name = 'choropleth',
    data = tot_unsheltered,
    columns = ['id', 'unsheltered'],
    key_on = 'feature.properties.id',
    fill_color = 'BuPu',
    fill_opacity = 0.7,
    line_opacity = 0.8, line_color = 'white',
    bins = bins,
    legend_name = 'Total Unsheltered (2017-2019)',
).add_to(m)

m.add_child(folium.LayerControl())

m.save('../notebooks/total_unsheltered.html')

## 2017 Unsheltered

In [6]:
df2 = df[df.year==2017].reset_index()

# to_json converts the gdf to json. json.loads converts it into dictionary 
# geo_data can have many more columns than just geometry. Need to include other columns if you want to have the HTML popup include info.
geo_data = json.loads(df2.set_index('id').to_json())

# Take what we want to map and turn it into a dictionary
# Can only include the key-value pair, the value you want to map, nothing more.
unshelt2017 = dict(zip(df2['id'].tolist(), df2['unsheltered'].tolist()))

df2.unsheltered.describe()

count     994.000000
mean       22.872718
std        53.524345
min         0.000000
25%         3.926000
50%        12.148500
75%        25.964500
max      1211.044000
Name: unsheltered, dtype: float64

In [7]:
m = folium.Map(
    location=[34.0536, -118.2427], 
    zoom_start=12, 
    tiles='cartodbpositron'
)

bins = [0, 10, 25, 50, 1400]

unsheltered = folium.Choropleth(
    geo_data = df2,
    name = 'choropleth',
    data = unshelt2017,
    columns = ['id', 'unsheltered'],
    key_on = 'feature.properties.id',
    fill_color = 'YlOrBr',
    fill_opacity = 0.7,
    line_opacity = 0.8, line_color = 'white',
    bins = bins,
    legend_name = '2017 Unsheltered',
).add_to(m)

m.add_child(folium.LayerControl())

m.save('../notebooks/unsheltered2017.html')

## 2018 Unsheltered

In [8]:
df3 = df[df.year==2018].reset_index()

geo_data = json.loads(df3.set_index('id').to_json())

unshelt2018 = dict(zip(df3['id'].tolist(), df3['unsheltered'].tolist()))

df3.unsheltered.describe()

count     993.000000
mean       21.909627
std        55.626741
min         0.000000
25%         3.831000
50%        10.783000
75%        23.443000
max      1341.172000
Name: unsheltered, dtype: float64

In [9]:
m = folium.Map(
    location=[34.0536, -118.2427], 
    zoom_start=12, 
    tiles='cartodbpositron'
)

bins = [0, 10, 25, 50, 1400]

unsheltered = folium.Choropleth(
    geo_data = df3,
    name = 'choropleth',
    data = unshelt2018,
    columns = ['id', 'unsheltered'],
    key_on = 'feature.properties.id',
    fill_color = 'YlOrBr',
    fill_opacity = 0.7,
    line_opacity = 0.8, line_color = 'white',
    bins = bins,
    legend_name = '2018 Unsheltered',
).add_to(m)

m.add_child(folium.LayerControl())

m.save('../notebooks/unsheltered2018.html')

## 2019 Unsheltered

In [10]:
df4 = df[df.year==2019].reset_index()

geo_data = json.loads(df4.set_index('id').to_json())

unshelt2019 = dict(zip(df4['id'].tolist(), df4['unsheltered'].tolist()))

df4.unsheltered.describe()

count     994.000000
mean       26.666236
std        61.657447
min         0.000000
25%         3.915000
50%        12.224000
75%        27.377750
max      1271.700000
Name: unsheltered, dtype: float64

In [11]:
m = folium.Map(
    location=[34.0536, -118.2427], 
    zoom_start=12, 
    tiles='cartodbpositron'
)

bins = [0, 10, 25, 50, 1400]

unsheltered = folium.Choropleth(
    geo_data = df4,
    name = 'choropleth',
    data = unshelt2019,
    columns = ['id', 'unsheltered'],
    key_on = 'feature.properties.id',
    fill_color = 'YlOrBr',
    fill_opacity = 0.7,
    line_opacity = 0.8, line_color = 'white',
    bins = bins,
    legend_name = '2019 Unsheltered',
).add_to(m)

m.add_child(folium.LayerControl())

m.save('../notebooks/unsheltered2019.html')